In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [30]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 30)

In [31]:
driver.get(
    "https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfPCD/classification.cfm"
)

# ✅ Correct locator from your screenshot
review_panel_dropdown = wait.until(
    EC.presence_of_element_located((By.NAME, "panel"))
)




StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
#0 0x568d0d61814a <unknown>
#1 0x568d0d0b5b80 <unknown>
#2 0x568d0d0c8c8b <unknown>
#3 0x568d0d0c7a42 <unknown>
#4 0x568d0d0bd209 <unknown>
#5 0x568d0d0bb2e3 <unknown>
#6 0x568d0d0beea8 <unknown>
#7 0x568d0d0bef33 <unknown>
#8 0x568d0d106b12 <unknown>
#9 0x568d0d107271 <unknown>
#10 0x568d0d0faf93 <unknown>
#11 0x568d0d12cefd <unknown>
#12 0x568d0d0faa2a <unknown>
#13 0x568d0d12d09e <unknown>
#14 0x568d0d15314a <unknown>
#15 0x568d0d12cca3 <unknown>
#16 0x568d0d0f8f08 <unknown>
#17 0x568d0d0fa071 <unknown>
#18 0x568d0d5e1b5b <unknown>
#19 0x568d0d5e5ae2 <unknown>
#20 0x568d0d5cd967 <unknown>
#21 0x568d0d5e66d4 <unknown>
#22 0x568d0d5b1c7f <unknown>
#23 0x568d0d606cd8 <unknown>
#24 0x568d0d606ea9 <u

In [33]:
select = Select(review_panel_dropdown)

In [35]:
reviewpanel = []

review_panel_dropdown = wait.until(
    EC.presence_of_element_located((By.NAME, "panel"))
)
select = Select(review_panel_dropdown)

for option in select.options:
    text = option.text.strip()
    if text:                       # skip empty option
        reviewpanel.append(text)

reviewpanel


['Anesthesiology',
 'General Hospital',
 'Ear Nose & Throat',
 'General & Plastic Surgery',
 'Immunology',
 'Ophthalmic',
 'Radiology',
 'Cardiovascular',
 'Gastroenterology/Urology',
 'Microbiology',
 'Orthopedic',
 'Clinical Chemistry',
 'Neurology',
 'Pathology',
 'Toxicology',
 'Dental',
 'Hematology',
 'Obstetrics/Gynecology',
 'Physical Medicine',
 'Molecular Genetics']

In [36]:
all_results = []


In [37]:
def click_search():
    search_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//select[@name='panel']/ancestor::form//input[@type='submit']")
        )
    )
    search_button.click()


In [43]:
def extract_results_table(review_panel_name):
    # Locate the main results table
    main_table = wait.until(
        EC.presence_of_element_located(
            (By.XPATH, "//table[contains(@class,'pcd-results-table') or @width='850']")
        )
    )

    # Actual data rows have bgcolor attribute
    rows = main_table.find_elements(
        By.XPATH, ".//tr[@bgcolor]"
    )

    for row in rows:
        cols = row.find_elements(By.XPATH, "./td")

        if len(cols) < 4:
            continue

        product_code = cols[0].text.strip()

        # Device name is inside a nested table
        device = cols[1].text.strip()

        regulation_number = cols[2].text.strip()
        device_class = cols[3].text.strip()

        all_results.append({
            "Review Panel": review_panel_name,
            "Product Code": product_code,
            "Device": device,
            "Regulation Number": regulation_number,
            "Device Class": device_class
        })


In [44]:
def click_search():
    search_button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//select[@name='panel']/ancestor::form//input[@type='submit']")
        )
    )
    search_button.click()


In [45]:
def go_to_new_search():
    new_search = wait.until(
        EC.element_to_be_clickable((By.LINK_TEXT, "New Search"))
    )
    new_search.click()

    wait.until(EC.presence_of_element_located((By.NAME, "panel")))


In [46]:
for panel_name in reviewpanel:
    print(f"Searching Review Panel: {panel_name}")

    # Select review panel
    review_panel_dropdown = wait.until(
        EC.presence_of_element_located((By.NAME, "panel"))
    )
    select = Select(review_panel_dropdown)
    select.select_by_visible_text(panel_name)

    # Click Search
    click_search()

    # Wait for results
    wait.until(EC.presence_of_element_located((By.XPATH, "//table")))
    time.sleep(1)

    # Extract table
    extract_results_table(panel_name)

    # Go back correctly
    go_to_new_search()


Searching Review Panel: Anesthesiology


TimeoutException: Message: 
Stacktrace:
#0 0x5d3d72dfe14a <unknown>
#1 0x5d3d7289bb80 <unknown>
#2 0x5d3d728ed0e9 <unknown>
#3 0x5d3d728ed271 <unknown>
#4 0x5d3d7293bde4 <unknown>
#5 0x5d3d72912efd <unknown>
#6 0x5d3d7293914a <unknown>
#7 0x5d3d72912ca3 <unknown>
#8 0x5d3d728def08 <unknown>
#9 0x5d3d728e0071 <unknown>
#10 0x5d3d72dc7b5b <unknown>
#11 0x5d3d72dcbae2 <unknown>
#12 0x5d3d72db3967 <unknown>
#13 0x5d3d72dcc6d4 <unknown>
#14 0x5d3d72d97c7f <unknown>
#15 0x5d3d72deccd8 <unknown>
#16 0x5d3d72decea9 <unknown>
#17 0x5d3d72dfcfc6 <unknown>
#18 0x7e4ccd494ac3 <unknown>
